#HIDS-506 Final Project: Exploratory Data Analysis and Data Cleaning (admissions.csv and patients.csv)

## Packages



We will start by loading some of the packages that will help us 
organize and visualize the data. Other packages will be loaded as necessary.

In [ ]:
#Import packages
from collections import Counter
import pandas as pd
import numpy as np
import collections, numpy
import mlxtend
import matplotlib 
from matplotlib import pyplot 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (10, 10)


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc

## Dataset

**Research Question:** Can machine learning (ML) provide a data-driven alternative to current emergency care triage and produce an accurate patient acuity score based on minimal patient information upon admission to the Emergency Department (ED), thus reducing human-biased medical decisions and preserving clinician time, attention, and ED workflow? 

MIMIC-IV 2.2 Admissions Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **language:** Spoken language of patient
* **ED Registration Time:** Time of patient registration in the ED (datetime)
* **ED Out-Time:** Time of patient discharge/transfer
* **Insurance:** Medicare, Medicaid, or Other

MIMIC-IV 2.2 Patients Dataset
* **Subject ID:** Unique patient identification number (will be dropped from feature set; used to merge datasets)
* **Anchor Age:** Age of patient during anchor year (years) (de-identified encoding)
* **Anchor Year:** Year range patient was seen in the ED (years) (de-identified encoding)



Exploring the datasets:

In [ ]:
#from google.colab import 
#drivedrive.mount('/content/drive')

In [ ]:
#mounting google drive to access files 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### MIMIC-IV Admissions Dataset

In [ ]:
# admissions file includes data from MIMIC-IV, patients will only be included if seen in the ED
# defining admissions file path 
admissions_file = '/content/drive/MyDrive/HIDS_506_FinalProject/data/admissions.csv'

# read in admissions file to dataframe
admissions_df = pd.read_csv(admissions_file)

In [ ]:
# visualizing the first few rows of admissions.csv
admissions_df.head()

subject_id   hadm_id            admittime            dischtime deathtime  \
0    10000032  22595853  2180-05-06 22:23:00  2180-05-07 17:15:00       NaN   
1    10000032  22841357  2180-06-26 18:27:00  2180-06-27 18:49:00       NaN   
2    10000032  25742920  2180-08-05 23:44:00  2180-08-07 17:50:00       NaN   
3    10000032  29079034  2180-07-23 12:35:00  2180-07-25 17:55:00       NaN   
4    10000068  25022803  2160-03-03 23:16:00  2160-03-04 06:26:00       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P874LG  TRANSFER FROM HOSPITAL   
1        EW EMER.            P09Q6Y          EMERGENCY ROOM   
2        EW EMER.            P60CC5          EMERGENCY ROOM   
3        EW EMER.            P30KEH          EMERGENCY ROOM   
4  EU OBSERVATION            P51VDL          EMERGENCY ROOM   

  discharge_location insurance language marital_status   race  \
0               HOME     Other  ENGLISH        WIDOWED  WHITE   
1               HOME  Medicaid  ENGLISH        WIDOWED  WHITE   
2            HOSPICE  Medicaid  ENGLISH        WIDOWED  WHITE   
3               HOME  Medicaid  ENGLISH        WIDOWED  WHITE   
4                NaN     Other  ENGLISH         SINGLE  WHITE   

             edregtime            edouttime  hospital_expire_flag  
0  2180-05-06 19:17:00  2180-05-06 23:30:00                     0  
1  2180-06-26 15:54:00  2180-06-26 21:31:00                     0  
2  2180-08-05 20:58:00  2180-08-06 01:44:00                     0  
3  2180-07-23 05:54:00  2180-07-23 14:00:00                     0  
4  2160-03-03 21:55:00  2160-03-04 06:26:00                     0

In [ ]:
# value counts of admission type
admissions_df['admission_type'].value_counts()

EW EMER.                       149413
EU OBSERVATION                  94776
OBSERVATION ADMIT               52668
URGENT                          44691
SURGICAL SAME DAY ADMISSION     34231
DIRECT EMER.                    19554
DIRECT OBSERVATION              18707
ELECTIVE                        10565
AMBULATORY OBSERVATION           6626
Name: admission_type, dtype: int64

In [ ]:
# visualizing missing values
admissions_df.isnull().sum()

subject_id                   0
hadm_id                      0
admittime                    0
dischtime                    0
deathtime               422633
admission_type               0
admit_provider_id            4
admission_location           0
discharge_location      119155
insurance                    0
language                     0
marital_status            9233
race                         0
edregtime               131949
edouttime               131949
hospital_expire_flag         0
dtype: int64

In [ ]:
# 16 features and 431231 observations 
admissions_df.shape

(431231, 16)

In [ ]:
# dropping all missing ED registration times
admissions_df = admissions_df.dropna(subset=['edregtime'])

In [ ]:
admissions_df.shape

(299282, 16)

In [ ]:
# value count of admission visits
admissions_df['admission_type'].value_counts()

EW EMER.                       142676
EU OBSERVATION                  94301
OBSERVATION ADMIT               45445
URGENT                           8873
DIRECT OBSERVATION               5541
DIRECT EMER.                     2230
AMBULATORY OBSERVATION            103
ELECTIVE                           84
SURGICAL SAME DAY ADMISSION        29
Name: admission_type, dtype: int64

In [ ]:
# value counts of insurance types
admissions_df['insurance'].value_counts()

Other       151807
Medicare    116614
Medicaid     30861
Name: insurance, dtype: int64

In [ ]:
admissions_df.isnull().sum()

subject_id                   0
hadm_id                      0
admittime                    0
dischtime                    0
deathtime               292832
admission_type               0
admit_provider_id            3
admission_location           0
discharge_location       98970
insurance                    0
language                     0
marital_status            6005
race                         0
edregtime                    0
edouttime                    0
hospital_expire_flag         0
dtype: int64

In [ ]:
# keeping subject id, language, insuarnce, edregtime, edouttime
admissions_df = admissions_df[['subject_id','language', 'insurance', 'edregtime', 'edouttime']]

In [ ]:
# 5 features and 299282
admissions_df.shape

(299282, 5)

In [ ]:
# language is no longer relevant to the model as there is only one option 'English'
admissions_df['language'].value_counts()

ENGLISH    268699
?           30583
Name: language, dtype: int64

In [ ]:
# keeping subject id, insuarnce, edregtime, edouttime
admissions_df = admissions_df[['subject_id', 'insurance', 'edregtime', 'edouttime']]

### MIMIC-IV Patients Dataset

In [ ]:
# patients file includes data from MIMIC-IV, patients will only be included if seen in the ED
# defining patients file path 
patients_file = '/content/drive/MyDrive/HIDS_506_FinalProject/data/patients.csv'

# read in patients file to dataframe
patients_df = pd.read_csv(patients_file)

In [ ]:
patients_df.head()

subject_id gender  anchor_age  anchor_year anchor_year_group         dod
0    10000032      F          52         2180       2014 - 2016  2180-09-09
1    10000048      F          23         2126       2008 - 2010         NaN
2    10000068      F          19         2160       2008 - 2010         NaN
3    10000084      M          72         2160       2017 - 2019  2161-02-13
4    10000102      F          27         2136       2008 - 2010         NaN

In [ ]:
# 6 features and 299712 observations 
patients_df.shape

(299712, 6)

In [ ]:
# no missing values, dod (date of death) includes 270636 missing values which signifies 270636 patients are still alive 
patients_df.isnull().sum()

subject_id                0
gender                    0
anchor_age                0
anchor_year               0
anchor_year_group         0
dod                  270636
dtype: int64

In [ ]:
# Track patients who died at the hospital by admission event
patients_df['deceased'] = patients_df['dod'].notnull().map({True:1, False:0})

In [ ]:
patients_df.head()

subject_id gender  anchor_age  anchor_year anchor_year_group         dod  \
0    10000032      F          52         2180       2014 - 2016  2180-09-09   
1    10000048      F          23         2126       2008 - 2010         NaN   
2    10000068      F          19         2160       2008 - 2010         NaN   
3    10000084      M          72         2160       2017 - 2019  2161-02-13   
4    10000102      F          27         2136       2008 - 2010         NaN   

   deceased  
0         1  
1         0  
2         0  
3         1  
4         0

In [ ]:
# Create age categories
age_ranges = [(0, 13), (13, 36), (36, 56), (56, 100)]
for num, cat_range in enumerate(age_ranges):
  patients_df['anchor_age'] = np.where(patients_df['anchor_age'].between(cat_range[0],cat_range[1]),
                                       num,patients_df['anchor_age'])
age_dict = {0: 'newborn', 1: 'young_adult', 2: 'middle_adult', 3: 'senior'}
patients_df['anchor_age'] = patients_df['anchor_age'].replace(age_dict)

In [ ]:
patients_df.head()

subject_id gender    anchor_age  anchor_year anchor_year_group         dod  \
0    10000032      F  middle_adult         2180       2014 - 2016  2180-09-09   
1    10000048      F   young_adult         2126       2008 - 2010         NaN   
2    10000068      F   young_adult         2160       2008 - 2010         NaN   
3    10000084      M        senior         2160       2017 - 2019  2161-02-13   
4    10000102      F   young_adult         2136       2008 - 2010         NaN   

   deceased  
0         1  
1         0  
2         0  
3         1  
4         0

In [ ]:
patients_df['age_category'] = patients_df['anchor_age']

In [ ]:
patients_df = patients_df[['subject_id', 'age_category', 'deceased']]

In [ ]:
# saving cleaned dataframes to csv 
admissions_df.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/admissions_clean.csv')
patients_df.to_csv('/content/drive/MyDrive/HIDS_506_FinalProject/data/patients_clean.csv')